In [19]:
import pandas as pd
import numpy as np
import datetime as dt

In [47]:
# Reading in data
data = pd.read_csv('technicals.csv')

In [48]:
# Setting index to datetime and removing bad columns
data = data.set_index('timestamp').drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1).copy()

In [50]:
# Creating buy and sell signals
data['buy'] = np.where((data['low'] <= data['lower_band']) & (data['time'] > '10:30:00'), True, False)
data['sell'] = np.where((data['high'] >= data['ema_10']) & (data['time'] > "10:30:00"), True, False)

In [51]:
# Removing times when buy and sell are at the same time and isolating times when buy or sell
data = data[(data['buy'] & data['sell']) != True].copy()
data = data[(data['buy'] | data['sell']) == True].copy()

In [52]:
data

,open,high,low,close,volume,vwap,transactions,date,time,ema_5,ema_10,ema_20,atr,upper_band,lower_band,buy,sell
timestamp,,,,,,,,,,,,,,,,,
2024-01-02 10:36:00+00:00,190.10,190.100000,190.100,190.100,424.0,190.1000,20,2024-01-02,10:36:00,190.085601,190.096235,190.082277,0.037420,190.231958,189.932596,False,True
2024-01-02 10:43:00+00:00,190.15,190.150000,190.150,190.150,605.0,190.1525,46,2024-01-02,10:43:00,190.107067,190.106011,190.088727,0.038049,190.240924,189.936530,False,True
2024-01-02 10:45:00+00:00,190.12,190.120000,190.120,190.120,386.0,190.1167,31,2024-01-02,10:45:00,190.111378,190.108554,190.091705,0.037647,190.242293,189.941118,False,True
2024-01-02 10:46:00+00:00,190.12,190.120000,190.120,190.120,302.0,190.1149,25,2024-01-02,10:46:00,190.114252,190.110635,190.094400,0.035764,190.237458,189.951342,False,True
2024-01-02 10:59:00+00:00,189.94,189.940000,189.700,189.760,9144.0,189.8299,325,2024-01-02,10:59:00,189.956445,190.018230,190.049609,0.049952,190.249417,189.849800,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-04 15:51:00+00:00,222.08,222.100000,221.900,221.930,64938.0,221.9874,955,2024-11-04,15:51:00,222.012336,221.933942,221.864249,0.196431,222.649972,221.078526,False,True
2024-11-04 15:52:00+00:00,221.93,222.010000,221.879,221.940,63587.0,221.9418,982,2024-11-04,15:52:00,221.988224,221.935043,221.871464,0.193159,222.644101,221.098827,False,True
2024-11-04 15:53:00+00:00,221.94,221.963348,221.800,221.805,54768.0,221.8607,883,2024-11-04,15:53:00,221.927149,221.911399,221.865134,0.191669,222.631808,221.098459,False,True
